In [1]:
import os
import sys
import functools

sys.path.append(os.path.realpath('../..'))
import aoc
my_aoc = aoc.AdventOfCode(2018,22)
from grid import Grid

In [2]:
# map area_types
area_type = {
    0: '.',
    1: '=',
    2: '|'
}

# reverse map to risk_level
risk_level =  {v: k for k, v in area_type.items()}

def parse_input(lines):
    depth = int(lines[0].split(' ')[1])
    target = lines[1].split(' ')[1]
    x_pos, y_pos = [int(pos) for pos in target.split(',')]
    return depth, (x_pos, y_pos)

@functools.lru_cache(maxsize=None)
def get_geologic_index(pos, depth, target):
    # The region at 0,0 (the mouth of the cave) has a geologic index of 0.
    # The region at the coordinates of the target has a geologic index of 0.
    if pos in [(0,0), target]:
        return 0
    if pos[1] == 0:
        # If the region's Y coordinate is 0, the geologic index is its X coordinate times 16807.
        return pos[0] * 16807
    # If the region's X coordinate is 0, the geologic index is its Y coordinate times 48271.
    if pos[0] == 0:
        return pos[1] * 48271
    # Otherwise, the region's geologic index is the result of multiplying the erosion levels of the regions at X-1,Y and X,Y-1.
    return get_erosion_level((pos[0] - 1, pos[1]), depth, target) * get_erosion_level((pos[0], pos[1] - 1), depth, target)

@functools.lru_cache(maxsize=None)
def get_erosion_level(pos, depth, target):
    """
    Function to calculate erosion level for an area
    Args:
        pos: tuple() -  int(x), int(y)
        depth: int()
        target: tuple() -  int(x), int(y)
    
    Returns:
        erosion_level: int()
    """
    # A region's erosion level is its geologic index plus the cave system's depth, all modulo 20183.
    geologic_index = get_geologic_index(pos, depth, target)
    return (geologic_index + depth) % 20183

@functools.lru_cache(maxsize=None)
def get_area_type(pos, depth, target):
    """
    function to get area type
    Args:
        pos: tuple() -  int(x), int(y)
        depth: int()
        target: tuple() -  int(x), int(y)
    Returns:
        area_type: char() '.', '=', or '|'
    """
    # If the erosion level modulo 3 is 0, the region's type is rocky.
    # If the erosion level modulo 3 is 1, the region's type is wet.
    # If the erosion level modulo 3 is 2, the region's type is narrow.
    erosion_level = get_erosion_level(pos, depth, target)
    return area_type[erosion_level % 3]

def get_risk_level(grid, start, goal):
    """
    Function to calculate risk level
    Args:
        grid: Grid() current map
        start: tuple() start position x/y coordinates
        goal: tuple() end position x/y coordinates
    Returns:
        total: int() risk_level total
    """
    total = 0
    min_x = min(start[0], goal[0])
    min_y = min(start[1], goal[1])
    max_x = max(start[0], goal[0])
    max_y = max(start[1], goal[1])    
    for point in grid:
        if min_x <= point[0] <= max_x and min_y <= point[1] <= max_y:
            total += risk_level[grid.get_point(point)]
    return total


In [3]:
input_text = """depth: 510
target: 10,10"""
# input_text = """depth: 4845
# target: 6,770"""
my_grid = Grid('M', type="infinite", coordinate_system='screen', use_overrides=False)
print(my_grid)
depth, target = parse_input(input_text.splitlines())
print("test Cases:")
for point in [(0,0), (1,0), (0,1), (1,1), (10, 10)]:
    my_area_type = get_area_type(point, depth, target)
    print(f"{point}: {my_area_type}")



M
test Cases:
(0, 0): .
(1, 0): |
(0, 1): .
(1, 1): =
(10, 10): .


In [4]:
print(depth, target)
for x_pos in range(target[0] + 6):
    for y_pos in range(target[1] + 6):
        my_grid.set_point((x_pos, y_pos), get_area_type((x_pos, y_pos), depth, target))
my_grid.update()

risk_level = get_risk_level(my_grid, (0,0), target)
my_grid.cfg['use_overrides'] = True
my_grid.overrides = {
    (0,0): 'M',
    target: 'T'
}
print(my_grid)
print(f"Risk Level is {risk_level}")

4845 (6, 770)
M|.|=.|=|=.|
.=.||=||.=|.
==.|.|....=.
====|||=.=|=
=.=..====.|.
|..|==|||=||
|===.=.==.|=
|.|.===|===|
.|.=.|.|===.
.=|.=|===.|=
=||=.|..|=..
=.|===.||=..
==|.==||.==.
|==.==.===|=
|.||===.|.|=
...||===|==|
..|.==.=||..
.||..=|====|
=.|.|...|||=
=|........==
|=|======|.=
|=.||=.|||==
||.|..|...||
.|||==.|.==|
.|.|.|..=.|.
=|||=|.==.==
=||||..=.|=|
======....=.
||=||=|=|==|
|=|...|=.=..
|.=|.|==|.||
.|.||=.=|===
.===|=.||.|=
=...||=|===|
=|..|||.|=..
==.|.|=|.|=|
|=.|===|..=|
|=...=..===|
..=.|====.=.
.|..=||.|=|=
.=|.==|...=|
=|===|=||..=
===.||.|..|.
|||=.|......
||.==..|.=|.
|=.=|..|.|=.
.=..==.=||=.
....|=.=.|.=
=.=|==.===|=
=..|===|.|==
===||=|..===
|.|.==.|=||.
|==|.|=.|=.|
||==..|||=.|
..|..||===..
..==..=..|..
=|.|.=.|.===
====|.=.|=|=
=..||.||==||
|===|=|.===|
|.==|.|..||=
..|..|.|=.|.
...|..|||..=
...|==.||...
=..=..|||..|
=.|===.=||==
|.|..|=|.==|
|||.=.||..|=
|||==...=.==
.|=|===|=..|
.=|=|.|||=|=
=|==.|=..|=.
=|=.|.|..||=
=|=|=||||.|=
||=||.|||==|
|=|....|.==